In [37]:
# Goal state
goalSt = [ [1, 2, 3], [4, 5, 6], [7, 8, 0] ]

argmin = lambda ls: sorted(ls, key = lambda x: x[0])[0][1]
# argmin recebe uma tupla (x, s) de um valor e um estado.
# faz uma ordenação pelo valor, retornando o estado [1] da
# tupla com menor valor [0]

# Maps a Move to a coordinate
move2coord = {"Up":(-1, 0),
              "Down": (1, 0),
              "Left": (0, -1),
              "Right": (0, 1)}
pos = lambda mv : move2coord[mv]
# O dicionário retorna um vetor/direção do movimento, que será
# somado à posição do qual o agente partiu.

def dist(v, X, Y):
    """
    Calcula a distância entre v no estado dado por X e v no estado
    dado por Y (inicial e final).
    input:
        v(int) : Um número.
        X(array[][]): Um tabuleiro (o atual, p.ex.)
        Y(array[][]): Um tabuleiro (o estado final, p.ex)
    """
    x, y = absDiff( findNum(v, X), findNum(v, Y) )
    return x + y

def absDiff(x, y):
    return ( abs(x[0] - y[0]), abs(x[1] - y[1]) )

def inBound(x):
    if 0 <= x <= 2:
        return True
    return False

# Check if it is a goal state
def isGoal(st):
    s, t = st
    return s == goalSt

# For this puzzle, every state is feasible (There are no constraints)
def feasible(x):
    return True

def addTuple(x, y):
    return (x[0] + y[0], x[1] + y[1])

findZero = lambda x: findNum(0, x)
def findNum(v, X):
    """
    Acha as coordenadas do valor "v" no tabuleiro.
    input:
        v(int) : Valor buscado.
        X(array[][]) : O tabuleiro (matriz) em um dado estado.
    """
    for i, Xi in enumerate(X):
        for j, xij in enumerate(Xi):
            if(xij == v): return (i, j)
            
            
def swap(val, s):
    new_s = [si.copy() for si in s] # faz uma cópia dos elementos de s
    x1, y1 = findNum(val, s) # acha o valor val
    x2, y2 = findZero(s)     # acha o jogador
    # faz a troca
    new_s[x1][y1] = 0        
    new_s[x2][y2] = val
    return new_s

In [38]:
# Update the state
def move(direc, st):
    """
    input:
        direc: Direção do movimento. É obtido pelo dicionário move2coord
        st: Uma tupla (x,y) em que x é o estado atual e y é a sequência
        de movimentos até então feitos para se chegar em x.
    """
    s, t = st
    newX, newY = addTuple(findZero(s), pos(direc))
    # encontra a posicao do jogador (o zero) no estado atual "s"
    # retorna o vetor direcao da escolha "direc"
    # retorna a posicao final do jogador
    if inBound(newX) and inBound(newY): # verifica se a posição é válida
        val = s[newX][newY] # recupera o valor que atualmente
                            # esta na proxima posicao
    else:
        val = 0 # colocando 0, o jogador permanece na mesma posicao
        
    new_s = swap(val, s) # troca o valor da posicao do jogador
                         # (o zero) com o valor da proxima posicao
    return (new_s, t + [direc]) # retorna o novo estado com 
    # um acrescimo na lista de movimentos
    
# perform a move
def moves( sts ):
    """
    Para cada elemento na lista de estados sts,
    input:
        sts(list) : O estado do qual o movimento está partindo
    """
    new_sts = []
    for st in sts:
        choices = move2coord.keys() # retorna as escolhas possiveis
        for c in choices:
            new_sts.append(move(c, st)) # para cada escolha,
            # adiciono o resultado do movimento executado
    return new_sts
    
# Breadth-first search
def bfs( sts ):
    """
    sts é inicial uma tupla com o estado atual e uma lista vazia
    de movimentos.
    """
    if len(sts) == 0:
        print("Couldn't find a feasible solution")
        return ([], []), 0
    goal = list(filter(isGoal, sts))
    n = 0 # conta quantas iterações foram necessárias para chegar
          # até a solução
    while len(goal) == 0:
        # dado um estado, executa-se todos os movimentos possiveis
        # e escolhe-se os movimentos válidos (no caso, todos).
        # A árvore de decisões vai se abrindo gradualmente.
        sts = list(filter(feasible, moves(sts)))
        goal = list(filter(isGoal, sts))
        n = n + len(sts) # essa contagem está correta?
    
    return goal[0], n

In [41]:
# Define as heurísticas
def f1(st):
    """
    Soma o custo unitário de cada decisão feita com as distâncias 
    horizontais e verticais de cada peça até sua posição alvo.
    """
    s, t = st
    return len(t) + sum([dist(v, s, goalSt) for v in range(1, 9)])

def f2(st):
    """
    Soma o custo unitário de cada decisão feita com o número de
    peças fora do lugar.
    """
    s, t = st
    return len(t) + sum([s[i][j] != goalSt[i][j]
                        for i in range(3)
                        for j in range(3) ]) - 1

def f3(st):
    """
    Retorna o valor máximo das heurísticas anteriores.
    """
    return max(f1(st), f2(st))


def astar( sts, f ):
    """
    input:
        sts: o estado do qual a busca parte?
        f: A heurística utilizada.
    """
    if len(sts) == 0: # verifica a validade do estado
        print("Couldn't find a feasible solution") 
        return ([], []), 0
    
    goal = list( filter(isGoal, sts) )
    n = 0
    sold = sts[0] # estado anterior
    while len(goal) == 0:
        s = argmin(zip(map(f, sts), sts)) # seleciona o estado com
        # menor custo segundo a heurística
        sts = [si for si in sts if si[0] != s[0]] # não pega estados repetidos (?)
        sts += moves([s]) # adiciona todos os movimentos possiveis partindo de s
        sold = s # isso parece ser inutil
        goal = list(filter(isGoal, sts))
        n = n + len(moves([s])) # essa contagem está correta?
        
    return goal[0], n

In [42]:
s0 = [[1, 8, 2], [0, 4, 3], [7, 6, 5]]

# sB, nB = bfs([(s0, [])])
# print(f'Busca em largura atingiu o estado {sB[0]} com a sequência {sB[1]} em {nB} jogadas\n\n')

sA1, nA1 = astar([(s0, [])], f1)
print(f'A*-f1 atingiu o estado {sA1[0]} com a sequência {sA1[1]} em {nA1} jogadas\n\n')


A*-f1 atingiu o estado [[1, 2, 3], [4, 5, 6], [7, 8, 0]] com a sequência ['Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Down'] em 44 jogadas


